<p style="font-weight:bold;"> <span style="font-size: 36px"> Reinsurance</span> </p>

<p style="font-weight:bold;"> <span style="font-size: 21px"> Model</span> </p>

# Data Import

In [0]:
#!eval-notebook "DataImport"
Workspace.InitializeFrom(DataSource);
//ifrs17.Reset(Workspace) //This is not working

In [0]:
ifrs17 = new Ifrs17(Workspace, Scopes, Report);

In [0]:
var reportingYear = 2020; 
var reportingMonth = 12;

# Best Estimate



In [0]:
var pv = ifrs17.PresentValues;
pv.ReportingNode = "CH";
pv.ReportingPeriod = (reportingYear, reportingMonth);
pv.CurrencyType = CurrencyType.Contractual;
pv.ColumnSlices = new string[]{"Scenario","GroupOfContract", "AmountType"};
pv.DataFilter = new [] {("EconomicBasis", "L"),("Scenario","!ELR60A70"),("Scenario","!ELR60A80")};
pv.Scenario = "All";// null;//"LR70";//"All";
(await pv.ToReportAsync)

# Written Actuals

Aslo the Actuals for both cases are exactly the same.

In [0]:
var writtenActual = ifrs17.WrittenActuals;
writtenActual.ReportingNode = "CH";
writtenActual.ReportingPeriod = (reportingYear, reportingMonth);
writtenActual.ColumnSlices = new string[]{"Scenario", "GroupOfContract"};
writtenActual.DataFilter =  null;//new [] {("GroupOfContract", "RP1.1")};
writtenActual.Scenario = "All";
(await writtenActual.ToReportAsync) with {Height = 400}

# Actuarial Experience Adjustment

In [0]:
var experienceAdjustments = ifrs17.ExperienceAdjustments;
experienceAdjustments.ReportingNode = "CH";
experienceAdjustments.ReportingPeriod = (reportingYear, reportingMonth);
experienceAdjustments.ColumnSlices = new string[]{"Scenario","GroupOfContract"};//"GroupOfContract", "AmountType"
experienceAdjustments.RowSlices = new string[]{"AmountType","EstimateType"};//"GroupOfContract", "AmountType"
experienceAdjustments.DataFilter = null; //new []{("VariableType", AocTypes.CF), ("AmountType","!CDR")}; //new [] {("Scenario","!ELR60A70"),("Scenario","!ELR60A80")}; //new [] {("GroupOfContract", "DT1.1")};
experienceAdjustments.Scenario = "All";
(await experienceAdjustments.ToReportAsync)

# Contractual Service Margin / Loss Component

In [0]:
var allocatedTechnicalMargins = ifrs17.AllocatedTechnicalMargins;
allocatedTechnicalMargins.ReportingNode = "CH";
allocatedTechnicalMargins.ReportingPeriod = (reportingYear, reportingMonth);
allocatedTechnicalMargins.ColumnSlices = new string[]{"Scenario", "GroupOfContract", "EstimateType"};
allocatedTechnicalMargins.DataFilter = new [] {("Scenario","!ELR60A70"),("Scenario","!ELR60A80")}; //new [] {("Scenario", "!LR80")};
//allocatedTechnicalMargins.Scenario = null;
allocatedTechnicalMargins.Scenario = "All";
(await allocatedTechnicalMargins.ToReportAsync) with {Height = 700}

# LRC Actuarial


In [0]:
var actuarialLrc = ifrs17.ActuarialLrc;
actuarialLrc.ReportingNode = "CH";
actuarialLrc.ReportingPeriod = (reportingYear, reportingMonth);
actuarialLrc.ColumnSlices = new string[]{"Scenario","GroupOfContract","EstimateType"};
actuarialLrc.DataFilter = new [] {("Scenario","!ELR60A70"),("Scenario","!ELR60A80")};
actuarialLrc.Scenario = "All";
(await actuarialLrc.ToReportAsync) with {Height = 750}

# Financial Performance


In [0]:
var financialPerformance = ifrs17.FinancialPerformance;
financialPerformance.ReportingNode = "CH";
financialPerformance.ReportingPeriod = (reportingYear, reportingMonth);
financialPerformance.ColumnSlices = new string[]{"Scenario", "GroupOfContract"};
financialPerformance.RowSlices = new string[]{"AmountType"};
financialPerformance.DataFilter = new []{("Scenario","!LR70"),("Scenario","!ELR60A70")};//new []{("Scenario","!ELR60A70"),("Scenario","!ELR60A80")};
financialPerformance.Scenario = "All";
(await financialPerformance.ToReportAsync) with { Height = 900, GroupDefaultExpanded = 3}

# KPIs

- CSM NB => Value Creation 
- CSM AM => Value Creation 
- Insurance Revenue => business Growth
- TCI PnL => PnL with OCI = 0 
- Experience Variance => managing underling business